#Book Recommender Based on kNN with Book similarity

The following is the detailed description of the implementation of kNN book recommender with book similarity. In summary, the between-book similarity is calculated based on the titles, authors, and publishers of books. For a given user, the books with the largest similarities with the books that the user have purchased are recommended. The weight for each of title, author, and publisher similarities is determined through the optimization of recommender accuracy in validation set. 

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import scipy.sparse as sparse
import sklearn.metrics as metrics
import sys as sys



/Users/tyyoo/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


##1. Data Loading and Cleaning

Here we will only deal with the data in 'Domestic' `Catagory`.

In [3]:
testdata = pd.read_csv("Test_Train_Sets/Dom_test.csv")
traindata = pd.read_csv("Test_Train_Sets/Dom_train.csv")
valdata = pd.read_csv("Test_Train_Sets/Dom_validate.csv")

In [4]:
traindata.shape, testdata.shape, valdata.shape

((21726, 18), (1212, 18), (1818, 18))

21726, 1212, and 1818 transactions in train, test, and validation set, respectively

In [5]:
print '%d different books and %d different users in traindata' %(np.unique(traindata['Title']).shape[0],np.unique(traindata['ID']).shape[0])
print '%d different books and %d different users in testdata' %(np.unique(testdata['Title']).shape[0],np.unique(testdata['ID']).shape[0])
print '%d different books and %d different users in valdata' %(np.unique(valdata['Title']).shape[0],np.unique(valdata['ID']).shape[0])


4823 different books and 8635 different users in traindata
813 different books and 303 different users in testdata
1070 different books and 303 different users in valdata


Since we don't need all features in the data to calculate the similarity between books, I extracted only relavant information such as author, publisher, and title.

In [7]:
#dataframe that contains only useful informations
usefulcol = ['Title','ID','Category','Author','ISBN','Publisher']
dftrain = traindata[usefulcol]
dftest = testdata[usefulcol]
dfval = valdata[usefulcol]

In [9]:
title_train = np.unique(dftrain['Title'].values)
author_train = np.unique(dftrain['Author'].values)
publisher_train = np.unique(dftrain['Publisher'].values)
title_train_dict = {i:t for i,t in enumerate(title_train)}
author_train_dict = {i:t for i,t in enumerate(author_train)}
publisher_train_dict = {i:t for i,t in enumerate(publisher_train)}

In `Author` field, authors are listed as a single text, for example "Authors: Person A, Person B, and Person C/Translators: Person D". In order to figure out whether two books share the same author, we need to change this single text to a vector of authors, such as [Person A, Person B, Person C]. In this process, I omitted translators in author list. The function below does this job.

In [10]:
import re
def author_to_list(author):
    #split the list of authors in a single string into a numpy list of authors. remove brackets, '저', '역', '그림', '/', and etc
    #remove translater
    if author != author:
        newauthorlist = []
    else:
        split_list = np.array(['/'])
        mask = np.array([s in author for s in split_list])
        split_list = split_list[mask]
        
        for s in split_list:
            author = re.sub(s,' ..',author)
        authorlist = author.split(' ..')
        authorlist = np.array(filter(None,authorlist))
        
        translatormask = np.array([(' 역' in a) or (' 공역' in a) or 
                                   (' 편역' in a) or (' 감수' in a) or 
                                   (' 옮김' in a) or (' 등역' in a) or
                                   (' 엮음' in a) for a in authorlist])
        authorlist = authorlist[~translatormask]
        
        remove_str_list = np.array(['<','>',' 그림',' 저',' 공저',' 글', ' 등저',' 공편','편'])
        newauthorlist = np.array([])
        for astr in authorlist:
            alist = np.array(astr.split(','))
            for i,a in enumerate(alist):
                for s in remove_str_list:
                    a = re.sub(s,'',a)
                alist.flat[i]=a
            newauthorlist = np.append(newauthorlist,alist)
                
    return newauthorlist       

Now I'm constructing the dataframe of books. The information in this dataframe will be used to calculate the similarity between books.

In [13]:
#dataframe of all books (no duplicates)
dfbook = dftrain[['Author','Title','Publisher','ISBN']]
dfbook = dfbook[~dfbook.duplicated()]
dfbook = dfbook.reset_index()
dfbook.drop('index', axis=1, inplace=True)

##2. Calculate author/publisher/title similarity between books 

Get entire list of authors, and use this list to make a Nbooks X Nauthors matrix, called Xauthor. Each (i,j) element in the matrix takes 1 if j-th author in the list of authors is an author of i-th book. This takes long time, so saved the constructed matrix in file. 

In [14]:
author_entire_list = np.array([])
for a in author_train:
    author_entire_list = np.append(author_entire_list,author_to_list(a))
author_entire_list = np.unique(author_entire_list)
author_entire_list.shape

(2865,)

In [14]:
#Make binary author lists: Nbooks X Nauthors matrix
#Use author_entire_list for index-author matching
Xauthor = np.zeros((dfbook.index.shape[0],author_entire_list.shape[0]))
for ind in dfbook.index:
    alist = author_to_list(dfbook.iloc[ind]['Author'])
    for a in alist:
        aind = np.where(author_entire_list==a)[0][0]
        Xauthor[ind,aind] = 1

In [15]:
np.savetxt('Xauthor_Dom.txt',Xauthor)

Similarly, make a NbooksXNpublishers matrix, called Xpublisher, where each (i,j) element in the matrix takes 1 if j-th publisher in the list of publishers is the publisher of i-th book. The matrix is saved in a file as well.

In [15]:
#Make binary publisher lists: Nbooks X Npublihser matrix
#use publisher_train or publisher_train_dict for index matching
Xpublisher = np.zeros((dfbook.index.shape[0],publisher_train.shape[0]))
for ind in dfbook.index:
    pub = dfbook.iloc[ind]['Publisher']
    pind = np.where(publisher_train==pub)[0][0]
    Xpublisher[ind,pind] = 1

In [17]:
np.savetxt('Xpublisher_Dom.txt',Xpublisher)

Obtained Xauthor matrix is used to contruct NbooksXNbooks author similarity matrix. (i,j) element of the matrix takes 1 if i-th and j-th book in `bfbook` have the same author, and 0 otherwise. Publihser similarity matrix is obtained similarly.

In [21]:
simmat_author = Xauthor.dot(Xauthor.T)
simmat_publisher = Xpublisher.dot(Xpublisher.T)

The similarity between two authors or two publishers is defined to take either zero (different) or one (same), because how similar the names of two authors or two publishers shouldn't affect the similarity between books. On the other hand, the simlarity between two titles should be able to take any values between 0 and 1, because book title often implies the sub-category of a book, and some people have a preference in some sub-category and are likely to buy a book in the sub-category repeatedly. <br>
The similarity between two text title can be calculated using `SequenceMatcher` function in `difflib`. Using the function, I made the Nbooks X Nbooks similarity matrix, where (i,j) element takes a value between 0 and 1 that quantifies the similarity between the titles of i-th book and j-th book in `dfbook` dataframe.<br>
<b>This is the longest task in this algorithm.</b>

In [16]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [19]:
%%time
#Make similarity matrix for titles: Nbooks X Nbooks matrix
#Use dfbooks for index-title matching
Nbooks =dfbook.index.shape[0]
simmat_title = np.zeros((Nbooks,Nbooks))
for i in range(Nbooks):
    simmat_title[i,i] = 1.0
    for j in range(i+1,Nbooks):
        simmat_title[i,j] = similar(dfbook.iloc[i]['Title'],dfbook.iloc[j]['Title'])
        simmat_title[j,i] = simmat_title[i,j]


CPU times: user 1h 46min 27s, sys: 2min 30s, total: 1h 48min 57s
Wall time: 1h 54min 14s


In [20]:
np.savetxt('simmat_title_Dom.txt',simmat_title)

Now that we have title/author/publisher similarity matrices, we may combine them to make combined similarity, and based on that we can recommend few books to a given user that are the most similar to the books that the user has bought so far. The function below does that job.

In [23]:
def recommend_based_on_similarity(dftrain,userid,Nrec,simmat,dfbook=dfbook):
    #recommend Nrec books similar to the books that the chosen user has bought so far
    #input: 
    # dftrain: training data
    # userid: id of the user
    # Nrec: Number of books to be recommended
    # dfbook: the dataframes of books
    # simmat: Nbooks X Nbooks similarity matrix
    usertrans = dftrain[dftrain['ID']==userid]  #user transaction
    usertrans = usertrans[['Author','Title','Publisher','ISBN']]
    usertrans = usertrans[~usertrans.duplicated()]
    excludeind = np.array([])
    simvec = np.zeros(simmat.shape[1])
    for ind in usertrans.index:
        bookind = dfbook[(dfbook['Title']==usertrans[usertrans.index == ind]['Title'].values[0]) 
                         & (dfbook['Publisher']==usertrans[usertrans.index == ind]['Publisher'].values[0])].index[0]
        simvec = simvec+simmat[bookind,:]
        excludeind = np.append(excludeind,[bookind])
    simvec = simvec/usertrans.index.shape[0]
    dfsimvec = dfbook
    dfsimvec['Similarity'] = simvec

    dfsimvec = dfsimvec.iloc[filter(lambda x: x not in excludeind,dfsimvec.index)]
    return dfsimvec.sort('Similarity',ascending=False)[:Nrec]
    


And the function below uses the function `recommend_based_on_similarity` above to calculate the prediction accuracy, where the prediction accuracy $\Lambda$ is defined as:
$$\Lambda = \frac{1}{N} \sum^N_{u=1} \frac{|Y_u \cap P_u|}{|Y_u|}$$
where $Y_u$ is the set of books that a user $u$ in the test/validation set purchased, and $P_u$ is the set of books recommended to the user $u$, and $N$ is the number of users in the test/validation set.

In [45]:
def predict_percentage(dftrain,dftest,Nrec,weightdict,simmat_author=simmat_author,
                       simmat_title=simmat_title,simmat_publisher=simmat_publisher,dfbook=dfbook):
    simmat = weightdict['Author']*simmat_author+weightdict['Title']*simmat_title
    +weightdict['Publisher']*simmat_publisher
    testusers = np.unique(dftest['ID'].values)
    totalbooks = 0
    totalsuccess = 0
    for user in testusers:
        dfrec = recommend_based_on_similarity(dftrain,user,Nrec,simmat,dfbook=dfbook)
        dftestuser = dftest[dftest['ID']==user]
        totalbooks = totalbooks+Nrec
        for i in range(dfrec.shape[0]):
            if np.any((dftestuser['Title']==dfrec.iloc[i]['Title']) 
                & (dftestuser['Publisher']==dfrec.iloc[i]['Publisher'])):
                totalsuccess=totalsuccess+1
    return float(totalsuccess)/float(totalbooks)
            
        

In [27]:
%%time
weightdict = {'Author':0.4,'Title':0.5,'Publisher':0.1}
print predict_percentage(dftrain,dfval,6,weightdict)

0.0687568756876
CPU times: user 21.7 s, sys: 1.78 s, total: 23.5 s
Wall time: 25.2 s


##3. Optimize Similarity Weight - Minimize prediction error in validation set

Here I did grid search to find the opitmal title/author/publisher similarity weight that minimizes the prediction error in validation set.

In [28]:
%%time
stepsize = 0.05

Aws = np.arange(0,1,stepsize)

accuracy_list = []
for A in Aws:
    Tws = np.arange(0,1-A,stepsize)
    for T in Tws:
        P = 1-A-T
        weightdict = {'Author':A,'Publisher':P,'Title':T}
        percentage = predict_percentage(dftrain,dfval,6,weightdict)
        accuracy = weightdict
        accuracy['accuracy'] = percentage
        accuracy_list = accuracy_list+[accuracy]
        

CPU times: user 1h 30min 32s, sys: 3min 33s, total: 1h 34min 5s
Wall time: 1h 44min 34s


In [29]:
import json
with open('accuracy_dom.json', 'w') as fp:
    json.dump(accuracy_list, fp)


In [34]:
accuracy_df = pd.DataFrame(accuracy_list)
accuracy_df = accuracy_df.sort('accuracy',ascending=False)
accuracy_df

,Author,Publisher,Title,accuracy
100,0.25,0.25,0.50,0.069857
43,0.10,0.70,0.20,0.069857
82,0.20,0.40,0.40,0.069857
114,0.30,0.25,0.45,0.069857
117,0.30,0.10,0.60,0.069857
22,0.05,0.85,0.10,0.069857
42,0.10,0.75,0.15,0.069857
80,0.20,0.50,0.30,0.069857
63,0.15,0.55,0.30,0.069857
146,0.45,0.45,0.10,0.069307


As you can see, multiple combinations of weights achieve the maximum accuracy. So I took the average of those combinations as below:

In [37]:
bestweightdict = dict(accuracy_df[accuracy_df['accuracy']==accuracy_df['accuracy'].max()].mean())
print bestweightdict

{'Publisher': 0.48333333333333328, 'Title': 0.33333333333333331, 'accuracy': 0.069856985698569851, 'Author': 0.18333333333333338}


In case of Domestic Literature, I found that when the weights are 0.48 for publisher, 0.33 for title, and 0.18 for author, the prediction accuracy is the best.

## 4. Measuring Test Prediction Accuracy

First I merged training and validation set to have a new bigger training set. And update the similarity matrices I've got above to take into account the books in validation sets that are not in the original training set. 

In [39]:
#merge train and validation set
dftrainandval = pd.concat([dftrain,dfval])
dftrainandval = dftrainandval.reset_index()
dftrainandval.drop('index', axis=1, inplace=True)

#dataframe of all books in train and val data(no duplicates)
dfbookall = dftrainandval[['Author','Title','Publisher','ISBN']]
dfbookall = dfbookall[~dfbookall.duplicated()]
dfbookall = dfbookall.reset_index()
dfbookall.drop('index', axis=1, inplace=True)


In [40]:
author_entire_list_all = np.array([])
for a in np.unique(dfbookall['Author']):
    author_entire_list_all = np.append(author_entire_list_all,author_to_list(a))
author_entire_list_all = np.unique(author_entire_list_all)
author_entire_list_all.shape

(3022,)

In [41]:
#make new simmat for merged dataframe
#Make binary author lists: Nbooks X Nauthors matrix
#Use author_entire_list for index-author matching
Xauthor_all = np.zeros((dfbookall.index.shape[0],author_entire_list_all.shape[0]))
for ind in range(dfbookall.shape[0]):
    alist = author_to_list(dfbookall.iloc[ind]['Author'])
    for a in alist:
        aind = np.where(author_entire_list_all==a)[0][0]
        Xauthor_all[ind,aind] = 1
        
#Make binary publisher lists: Nbooks X Npublihser matrix
#use publisher_train or publisher_train_dict for index matching
publisher_list_all = np.unique(dfbookall['Publisher'])
Xpublisher_all = np.zeros((dfbookall.index.shape[0],publisher_list_all.shape[0]))
for ind in range(dfbookall.shape[0]):
    pub = dfbookall.iloc[ind]['Publisher']
    pind = np.where(publisher_list_all==pub)[0][0]
    Xpublisher_all[ind,pind] = 1

In [42]:
%%time
#Update similarity matrix for titles:
Nbooks_all =dfbookall.index.shape[0]
simmat_title_all = np.zeros((Nbooks_all,Nbooks_all))
Nbooks = dfbook.index.shape[0]
for i in range(Nbooks,Nbooks_all):
    for j in range(Nbooks):
        simmat_title_all[i,j] = similar(dfbookall.iloc[i]['Title'],dfbookall.iloc[j]['Title'])
        simmat_title_all[j,i] = simmat_title_all[i,j]
        
for i in range(Nbooks,Nbooks_all):
    simmat_title_all[i,i] = 1.0
    for j in range(i+1,Nbooks_all):
        simmat_title_all[i,j] = similar(dfbookall.iloc[i]['Title'],dfbookall.iloc[j]['Title'])
        simmat_title_all[j,i] = simmat_title_all[i,j]


CPU times: user 18min 54s, sys: 4.71 s, total: 18min 59s
Wall time: 24min 25s


In [43]:
simmat_author_all = Xauthor_all.dot(Xauthor_all.T)
simmat_publisher_all = Xpublisher_all.dot(Xpublisher_all.T)

<b>Now everything is ready to calculate the test error!!!<b>

In [46]:
#Estimate Test error
testerror = predict_percentage(dftrainandval,dftest,4,bestweightdict,
                               simmat_author=simmat_author_all,simmat_title=simmat_title_all,
                               simmat_publisher=simmat_publisher_all,dfbook=dfbookall)
print testerror

0.0255775577558
